# Исследование набора данных Gender Recognition by Voice

English version of this research: https://www.kaggle.com/elopatkin/gender-recognition-by-voice-en

Проведем исследование набора данных "Voice Gender". В нем различные характеристики голоса сопоставляются полу человека. Проверим насколько женский голос отличается от мужского и сможем ли мы создать модель, которая будет эффективно классифицировать голоса.

## Анализ данных

Посмотрим на пример данных из набора.

In [ ]:
import pandas as pd

df = pd.read_csv('../input/voicegender/voice.csv')
df.head()


В наборе данных имеется множество столбцов с числовыми характеристиками голоса, а также столбец "label", где указано к какому полу относится голос. Исследуем корреляцию различных параметров с полом.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

label = df['label'].map({ 'male': 0, 'female': 1 }, na_action='ignore')
corr_data = df.drop('label', axis=1).corrwith(label).abs().sort_values()

plt.figure(figsize=(12, 8))
sns.barplot(x=corr_data.values, y=corr_data.index)
plt.title('Correlation with gender')
plt.xlabel('correlation (abs)')
plt.show()

По графику видна высокая связь столбца "meanfun" с полом. Ниже представлены описания трех наиболее значимых столбцов:
* meanfun: "average of dominant frequency measured across acoustic signal"
* IQR: "interquantile range (in kHz)"
* Q25: "first quantile (in kHz)"

Нарисуем график распределения значений столбца "meanfun", чтобы понять его связь с полом.

In [ ]:
plt.figure(figsize=(10, 6))
sns.swarmplot(x=df['label'], y=df['meanfun'])
plt.title('meanfun distribution by gender')
plt.show()

Из графика выше видно, что среднее значение доминирующей частоты сигнала у мужчин ниже, чем у женщин. В большинстве случаев голос у мужчин ниже, чем у женщин.

Теперь посмотрим на две другие характеристики: IQR и Q25. Построим  график распределение обех величин.

In [ ]:
sns.jointplot(x=df['IQR'], y=df['Q25'], kind='kde')
plt.show()

Распределение этих величин говорит нам о наличии двух пересекающися кластеров. Скорее всего даже по лишь этим параметрам можно было бы построить более-менее приемлимую модель.

## Обучение модели 

Учитывая высокий уровень корреляции между полом и рядом голосовых характеристик, я решил использовать простую классифицирующую модель. При обучении будем использовать лишь наиболее коррелирующие параметры.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

y = df['label'].map({ 'male': 0, 'female': 1 }, na_action='ignore')
X = df[['meanfun', 'IQR', 'Q25', 'sp.ent', 'sd', 'sfm', 'centroid', 'meanfreq', 'median']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print('Accuracy is', metrics.accuracy_score(y_test, y_pred))

Ради интереса проверим насколько эффективной была бы модель, если не учитывать наиболее коррелирующее поле 'meanfun'.

In [ ]:
y = df['label'].map({ 'male': 0, 'female': 1 }, na_action='ignore')
X = df[['IQR', 'Q25', 'sp.ent', 'sd', 'sfm', 'centroid', 'meanfreq', 'median']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print('Accuracy is', metrics.accuracy_score(y_test, y_pred))

## Спасибо

Благодарю Вас за уделенное внимание. Это моя первая работа на kaggle. Буду рад вашим замечаниям и исправлениям.